# Pipeline Produto 1 — Base TSBio (RAW → Processado → Catálogo + Base Consolidada)

Este notebook substitui os scripts CLI e roda **todo o pipeline** em ambiente Jupyter.

## O que ele faz
1. **Processa** `data/Indicadores/**.csv` → `data/Indicadores_processado_por_tema/<Categoria>/*.csv`
2. Gera **Catálogo** → `outputs/catalogo_indicadores_tsbio.(csv|xlsx)`
3. Gera **Base Consolidada (long/tidy)** → `outputs/base_consolidada_tsbio.csv`

> **Padrão de nome de arquivo bruto**:  
> `FONTE - TEMA - RECORTE.csv` (RECORTE é opcional)  
> Ex.: `IBGE Censo Agro 2017 - Área média (ha) - Municípios.csv`


## 0) Configuração rápida

Ajuste apenas as variáveis abaixo (pastas de entrada/saída).  
Depois rode as células na ordem.


In [1]:
from pathlib import Path

# ====== AJUSTE AQUI (Windows) ======
DATA_DIR = Path(r"C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data")

# Pastas
ROOT_RAW = DATA_DIR / "Indicadores"  # brutos
OUT_PROCESSADO = DATA_DIR / "Indicadores_processado_por_tema"  # processados

# Dicionário oficial (coloque este arquivo em fas_tsbio\notebook\ ou no diretório do projeto)
# Opção 1 (recomendada): salvar dentro da pasta notebook
DICT_PATH = Path(r"C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\notebook\dicionario_nomes_oficial_tsbio.csv")

# Saídas (pode ficar dentro de data/outputs para manter tudo junto)
OUT_DIR = DATA_DIR / "outputs"

REQUIRE_FULL_COVERAGE = False  # se True, marca indicadores incompletos no relatório
EXPORT_EXCEL_PROCESSADO = False  # se True, exporta xlsx por indicador (mais lento)

# Arquivos de saída
OUT_BASE_CONSOLIDADA = OUT_DIR / "base_consolidada_tsbio.csv"
OUT_CATALOGO_CSV = OUT_DIR / "catalogo_indicadores_tsbio.csv"
OUT_CATALOGO_XLSX = OUT_DIR / "catalogo_indicadores_tsbio.xlsx"

# Cria pastas se não existirem
OUT_PROCESSADO.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Validações rápidas (para evitar rodar com caminho errado)
assert ROOT_RAW.exists(), f"ROOT_RAW não encontrado: {ROOT_RAW}"
assert DICT_PATH.exists(), f"DICT_PATH não encontrado: {DICT_PATH}"

ROOT_RAW, OUT_PROCESSADO, DICT_PATH, OUT_DIR

(WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/Indicadores_processado_por_tema'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/notebook/dicionario_nomes_oficial_tsbio.csv'),
 WindowsPath('C:/Users/luiz.felipe/Desktop/FLP/MapiaEng/GitHub/fas_tsbio/data/outputs'))

In [2]:
import re
import unicodedata
from collections import defaultdict
from typing import Dict, List, Tuple

import pandas as pd
from tqdm import tqdm


In [3]:
# =============================================================================
# CONFIG TSBio (6 territórios)
# =============================================================================

TSBIO = [
    {"territorio_id": 1, "territorio_nome": "Altamira", "CD_MUN": ["1500602","1500859","1501725","1504455","1505486","1507805","1508159","1508357"]},
    {"territorio_id": 2, "territorio_nome": "Macapá", "CD_MUN": ["1600212","1600303","1600253","1600238","1600535","1600600","1600154","1600055"]},
    {"territorio_id": 3, "territorio_nome": "Portel", "CD_MUN": ["1503101","1504505","1505809","1501105"]},
    {"territorio_id": 4, "territorio_nome": "Juruá-Tefé", "CD_MUN": ["1301654","1301803","1301407","1301506","1301951","1301001","1304203","1302207","1304260","1300029"]},
    {"territorio_id": 5, "territorio_nome": "Rio Branco-Brasiléia", "CD_MUN": ["1200401","1200708","1200252","1200104","1200054","1200138","1200807","1200450","1200013","1200385","1200179"]},
    {"territorio_id": 6, "territorio_nome": "Salgado-Bragantino", "CD_MUN": ["1508209","1508035","1507961","1507474","1507466","1507409","1507102","1506906","1506609","1506203","1506112","1506104","1505601","1505007","1504406","1504307","1504109","1503200","1502905","1502608","1502202","1501709","1501600","1500909"]},
]

CATEGORIA_TO_DIMENSAO = {
    "População": "socioeconômica",
    "Educação": "socioeconômica",
    "Domicílios": "socioeconômica",
    "Trabalho e Renda": "socioeconômica",
    "Mercado de Trabalho": "socioeconômica",
    "Economia": "socioeconômica",
    "Religião": "socioeconômica",
    "Indígenas": "socioeconômica",
    "Quilombola": "socioeconômica",
    "Índices": "socioeconômica",
    "Desmatamento": "ambiental",
    "Uso e Cobertura do Solo": "ambiental",
    "Fundiário": "ambiental",
    "Agropecuária": "produtiva",
    "Entorno Domicílios": "infraestrutura",
    "Assistência Social": "políticas_públicas",
    "Vulnerabilidade": "vulnerabilidades",
    "Favelas e Comunidades Urbanas": "vulnerabilidades",
}

# Separadores
CSV_SEP_DEFAULT = ";"
SEPS_CANDIDATES = [";", ",", "\t"]
OUT_SEP = ";"
OUT_ENCODING = "utf-8-sig"


In [4]:
# =============================================================================
# Utilidades — normalização / dicionário / CSV
# =============================================================================

def safe_filename(name: str) -> str:
    name = re.sub(r'[\\/:*?"<>|]+', "_", (name or "").strip())
    name = re.sub(r"\s+", " ", name).strip()
    return name or "_"

def slugify(s: str) -> str:
    s = (s or "").strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def zfill_mun(x) -> str:
    x = re.sub(r"\D+", "", str(x))
    return x.zfill(7) if x else ""

def parse_parts_from_filename(filename: str) -> Tuple[str, str, str]:
    """Extrai (fonte, tema, recorte) via separador seguro ' - '."""
    name = filename[:-4] if filename.lower().endswith(".csv") else filename
    parts = [p.strip() for p in name.split(" - ") if p.strip()]
    fonte = parts[0] if len(parts) >= 1 else ""
    tema = parts[1] if len(parts) >= 2 else (parts[0] if parts else "")
    recorte = parts[2] if len(parts) >= 3 else ""
    return fonte, tema, recorte

def build_indicador_id(categoria: str, fonte: str, tema: str, recorte: str = "") -> str:
    base = f"{slugify(categoria)}__{slugify(fonte)}__{slugify(tema)}"
    if recorte and slugify(recorte) not in ("municipios", "municípios"):
        base = f"{base}__{slugify(recorte)}"
    return base

def detect_csv_sep(path: Path, encoding: str) -> str:
    try:
        with open(path, "r", encoding=encoding, errors="replace") as f:
            header = f.readline()
            if header.lower().startswith("sep="):
                header = f.readline()
    except Exception:
        return CSV_SEP_DEFAULT

    if not header:
        return CSV_SEP_DEFAULT

    counts = {s: header.count(s) for s in SEPS_CANDIDATES}
    best = max(counts, key=counts.get)
    return best if counts[best] > 0 else CSV_SEP_DEFAULT

def read_csv_local(path: Path) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8-sig", "latin1"):
        try:
            sep = detect_csv_sep(path, enc)
            return pd.read_csv(path, sep=sep, encoding=enc)
        except Exception as e:
            last_err = e
            continue
    raise last_err

def load_dictionary(path: Path) -> Dict[str, str]:
    """Carrega dicionário oficial: normalized_synonym -> canonical."""
    if not path.exists():
        return {}
    df = pd.read_csv(path, encoding="utf-8-sig")
    syn_map: Dict[str, str] = {}
    for _, r in df.iterrows():
        canonical = str(r["canonical"]).strip()
        syns = str(r.get("synonyms", "") or "").split("|")
        for s in syns + [str(r.get("label_pt", "") or "")]:
            ss = slugify(s)
            if ss:
                syn_map[ss] = canonical
    return syn_map

_UNIT_SUFFIX_MAP = {
    "%": "perc",
    "percent": "perc",
    "porcentagem": "perc",
    "ha": "ha",
    "hectare": "ha",
    "hectares": "ha",
    "km2": "km2",
    "km²": "km2",
    "m2": "m2",
    "m²": "m2",
    "r$": "rs",
    "rs": "rs",
    "reais": "rs",
    "pessoas": "pessoas",
    "pessoa": "pessoas",
}

def _unit_to_suffix(unit_raw: str) -> str:
    u = (unit_raw or "").strip().lower()
    if not u:
        return ""
    u = u.replace(" ", "")
    u = u.replace("R$", "r$").replace("²", "2")
    if "/" in u:
        left, right = u.split("/", 1)
        left_s = _UNIT_SUFFIX_MAP.get(left, slugify(left))
        right_s = _UNIT_SUFFIX_MAP.get(right, slugify(right))
        return f"{left_s}_por_{right_s}".strip("_")
    return _UNIT_SUFFIX_MAP.get(u, slugify(u))

_UNIT_PATTERN = re.compile(r"^(.*?)[\s]*[\(\[\{]([^\)\]\}]+)[\)\]\}]\s*$")

def normalize_column_name(col: str, syn_map: Dict[str, str]) -> str:
    """
    1) Se bater no dicionário oficial -> canonical
    2) Senão: snake_case + unidade no sufixo se houver (ex.: '(ha)' -> '_ha')
    """
    col = (col or "").strip()
    if not col:
        return col

    s = slugify(col)
    if s in syn_map:
        return syn_map[s]

    m = _UNIT_PATTERN.match(col)
    unit_suffix = ""
    base = col
    if m:
        base = m.group(1).strip()
        unit_suffix = _unit_to_suffix(m.group(2).strip())

    base_slug = slugify(base)
    if unit_suffix and not base_slug.endswith("_" + unit_suffix):
        base_slug = f"{base_slug}_{unit_suffix}"
    return base_slug or s


## 1) Processar brutos → processados

Gera um CSV por indicador, já com colunas padronizadas e metadados adicionados.


In [5]:
def processar_indicadores(
    root_raw: Path,
    out_dir: Path,
    dict_path: Path,
    require_full_coverage: bool = False,
) -> pd.DataFrame:
    syn_map = load_dictionary(dict_path)

    expected_muns = set()
    mun_to_tsbio: Dict[str, Tuple[int, str]] = {}
    for t in TSBIO:
        tid = int(t["territorio_id"])
        tname = str(t["territorio_nome"])
        for m in t["CD_MUN"]:
            mm = zfill_mun(m)
            expected_muns.add(mm)
            mun_to_tsbio[mm] = (tid, tname)

    csv_files = sorted(root_raw.rglob("*.csv"))
    print(f"CSV brutos encontrados: {len(csv_files)} em {root_raw}")

    buckets: Dict[Tuple[str, str, str, str], List[pd.DataFrame]] = defaultdict(list)
    errors = []
    missing_cod_mun = []

    for p in tqdm(csv_files, desc="Lendo CSVs brutos"):
        try:
            rel = p.relative_to(root_raw)
            categoria = rel.parts[0] if len(rel.parts) >= 2 else "(raiz)"
        except Exception:
            categoria = "(raiz)"

        fonte, tema, recorte = parse_parts_from_filename(p.name)
        indicador_id = build_indicador_id(categoria, fonte, tema, recorte)

        try:
            df = read_csv_local(p)
        except Exception as e:
            errors.append((str(p), str(e)))
            continue

        col_map = {c: normalize_column_name(c, syn_map) for c in df.columns}
        df = df.rename(columns=col_map)

        if "cod_municipio" not in df.columns:
            possible = [c for c in df.columns if c.endswith("municipio") and "cod" in c]
            if possible:
                df = df.rename(columns={possible[0]: "cod_municipio"})

        if "cod_municipio" not in df.columns:
            missing_cod_mun.append(str(p))
            continue

        df["cod_municipio"] = df["cod_municipio"].apply(zfill_mun)
        df = df[df["cod_municipio"].astype(str).str.len() > 0].copy()

        # filtro TSBio
        df = df[df["cod_municipio"].isin(expected_muns)].copy()
        if df.empty:
            continue

        # metadados
        df["arquivo_origem"] = p.name
        df["categoria"] = categoria
        df["fonte"] = fonte
        df["tema"] = tema
        df["recorte"] = recorte
        df["dimensao_tdr"] = CATEGORIA_TO_DIMENSAO.get(categoria, "outros")
        df["indicador_id"] = indicador_id

        # mapeamento TSBio
        df["territorio_id"] = df["cod_municipio"].map(lambda x: mun_to_tsbio.get(x, (None, None))[0])
        df["territorio_nome"] = df["cod_municipio"].map(lambda x: mun_to_tsbio.get(x, (None, None))[1])
        df = df[df["territorio_id"].notna()].copy()

        buckets[(categoria, fonte, tema, recorte)].append(df)

    report = []
    for (categoria, fonte, tema, recorte), dfs in buckets.items():
        big = pd.concat(dfs, ignore_index=True)
        present = set(big["cod_municipio"].dropna().unique())
        missing = sorted(list(expected_muns - present))

        status = "ok"
        if require_full_coverage and missing:
            status = "incompleto"

        cat_dir = out_dir / safe_filename(categoria)
        cat_dir.mkdir(parents=True, exist_ok=True)

        name_parts = [fonte, tema] + ([recorte] if recorte else [])
        out_base = safe_filename(" - ".join(name_parts))
        out_csv_path = cat_dir / f"{out_base}.csv"

        first_cols = [
            "indicador_id", "territorio_id", "territorio_nome", "categoria",
            "fonte", "tema", "recorte", "dimensao_tdr", "cod_municipio",
            "municipio_nome", "sigla_uf", "ano", "mes", "classe", "observacao", "arquivo_origem"
        ]
        cols = [c for c in first_cols if c in big.columns] + [c for c in big.columns if c not in first_cols]
        big = big[cols]

        big.to_csv(out_csv_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING)

        report.append({
            "categoria": categoria,
            "fonte": fonte,
            "tema": tema,
            "recorte": recorte,
            "indicador_id": big["indicador_id"].iloc[0],
            "dimensao_tdr": big["dimensao_tdr"].iloc[0],
            "status": status,
            "arquivo_csv": str(out_csv_path),
            "linhas": len(big),
            "n_presentes": len(present),
            "n_esperados": len(expected_muns),
            "faltando_cod_municipio": ",".join(missing) if missing else "",
            "n_colunas": len(big.columns),
        })

    rep_df = pd.DataFrame(report).sort_values(["status", "categoria", "fonte", "tema"])
    rep_path = out_dir / "_relatorio_validacao.csv"
    rep_df.to_csv(rep_path, index=False, encoding=OUT_ENCODING)

    if errors:
        pd.DataFrame(errors, columns=["arquivo", "erro"]).to_csv(out_dir / "_erros_leitura.csv", index=False, encoding=OUT_ENCODING)
    if missing_cod_mun:
        pd.DataFrame({"arquivo": missing_cod_mun}).to_csv(out_dir / "_sem_coluna_cod_municipio.csv", index=False, encoding=OUT_ENCODING)

    print(f"✅ Processamento concluído. Relatório: {rep_path}")
    return rep_df


In [6]:
rep_df = processar_indicadores(
    root_raw=ROOT_RAW,
    out_dir=OUT_PROCESSADO,
    dict_path=DICT_PATH,
    require_full_coverage=REQUIRE_FULL_COVERAGE,
)
rep_df.head(10)

CSV brutos encontrados: 9773 em C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\Indicadores


Lendo CSVs brutos: 100%|██████████| 9773/9773 [03:25<00:00, 47.55it/s] 


✅ Processamento concluído. Relatório: C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\Indicadores_processado_por_tema\_relatorio_validacao.csv


,categoria,fonte,tema,recorte,indicador_id,dimensao_tdr,status,arquivo_csv,linhas,n_presentes,n_esperados,faltando_cod_municipio,n_colunas
0,Agropecuária,IBGE PAM 2024,Lavoura Permanente,Municípios,agropecuaria__ibge_pam_2024__lavoura_permanente,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,125970,65,65,,20
1,Agropecuária,IBGE PAM 2024,Lavoura Temporaria,Municípios,agropecuaria__ibge_pam_2024__lavoura_temporaria,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,109395,65,65,,20
2,Agropecuária,IBGE PEVS 2024,Produção Extração Vegetal,BR,agropecuaria__ibge_pevs_2024__producao_extraca...,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,10471,65,65,,16
3,Agropecuária,IBGE PEVS 2024,Produção Silvicultura,BR,agropecuaria__ibge_pevs_2024__producao_silvicu...,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,69,5,65,"1200013,1200054,1200104,1200138,1200179,120025...",18
4,Agropecuária,IBGE PPM 2024,Efetivo Rebanhos,BR,agropecuaria__ibge_ppm_2024__efetivo_rebanhos__br,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,15838,65,65,,14
5,Agropecuária,IBGE PPM 2024,Producão Aquicultura,BR,agropecuaria__ibge_ppm_2024__producao_aquicult...,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,1892,54,65,"1302207,1501105,1502608,1503101,1504505,150500...",15
6,Agropecuária,IBGE PPM 2024,Produção Origem Animal,BR,agropecuaria__ibge_ppm_2024__producao_origem_a...,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,7171,65,65,,16
7,Agropecuária,IBGE PPM 2024,Produção Pecuária,BR,agropecuaria__ibge_ppm_2024__producao_pecuaria...,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,2630,65,65,,14
8,Agropecuária,PRONAF,Cobertura do PRONAF,Municípios,agropecuaria__pronaf__cobertura_do_pronaf,produtiva,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,65,65,65,,15
9,Assistência Social,Observatório Cadastro Único,Cadastro Único,Municípios,assistencia_social__observatorio_cadastro_unic...,políticas_públicas,ok,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,65,65,65,,21


## 2) Gerar base consolidada (long/tidy)

Transforma os CSVs processados em uma base única no formato long:
- `variavel`, `valor`, `valor_num`, `unidade`


In [7]:
UNIT_SUFFIX_TO_UNIT = {
    "perc": "%",
    "ha": "ha",
    "km2": "km²",
    "m2": "m²",
    "rs": "R$",
    "pessoas": "pessoas",
}

def infer_unidade_from_variavel(variavel: str) -> str:
    if not variavel:
        return ""
    v = str(variavel).strip().lower()
    suf = v.split("_")[-1]
    return UNIT_SUFFIX_TO_UNIT.get(suf, "")

def identificar_colunas_valor(df: pd.DataFrame) -> List[str]:
    excluir = {
        "indicador_id", "territorio_id", "territorio_nome", "categoria",
        "fonte", "tema", "recorte", "dimensao_tdr",
        "cod_municipio", "municipio_nome", "sigla_uf",
        "ano", "mes", "classe", "observacao", "arquivo_origem",
    }
    cols_valor = []
    for col in df.columns:
        if col in excluir:
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            cols_valor.append(col)
            continue
        if df[col].dtype == "object":
            s = df[col].astype(str)
            numeric = pd.to_numeric(
                s.str.replace(".", "", regex=False)
                 .str.replace(",", ".", regex=False)
                 .str.replace("%", "", regex=False)
                 .str.replace(" ", "", regex=False),
                errors="coerce",
            )
            if numeric.notna().mean() > 0.5:
                cols_valor.append(col)
    return cols_valor

def transformar_para_long(df: pd.DataFrame) -> pd.DataFrame:
    id_cols = [c for c in [
        "territorio_id","territorio_nome","cod_municipio","municipio_nome","sigla_uf",
        "ano","mes","indicador_id","tema","categoria","dimensao_tdr","fonte",
        "recorte","classe","observacao","arquivo_origem"
    ] if c in df.columns]

    value_cols = identificar_colunas_valor(df)
    if not value_cols:
        return pd.DataFrame()

    melted = pd.melt(df, id_vars=id_cols, value_vars=value_cols, var_name="variavel", value_name="valor")

    s = melted["valor"].astype(str)
    melted["valor_num"] = pd.to_numeric(
        s.str.replace(".", "", regex=False)
         .str.replace(",", ".", regex=False)
         .str.replace("%", "", regex=False)
         .str.replace(" ", "", regex=False),
        errors="coerce",
    )

    melted["unidade"] = melted["variavel"].map(infer_unidade_from_variavel)

    order = [
        "territorio_id","territorio_nome","cod_municipio","municipio_nome","sigla_uf",
        "ano","mes",
        "indicador_id","tema","categoria","dimensao_tdr","fonte","recorte",
        "variavel","valor","valor_num","unidade",
        "classe","observacao","arquivo_origem",
    ]
    cols = [c for c in order if c in melted.columns] + [c for c in melted.columns if c not in order]
    return melted[cols]

def gerar_base_consolidada(root_processado: Path, out_path: Path) -> pd.DataFrame:
    files = sorted(root_processado.rglob("*.csv"))
    print(f"Processados encontrados: {len(files)} em {root_processado}")

    out_parts = []
    for p in tqdm(files, desc="Consolidando"):
        try:
            df = read_csv_local(p)
        except Exception:
            continue

        df_long = transformar_para_long(df)
        if not df_long.empty:
            out_parts.append(df_long)

    if not out_parts:
        raise RuntimeError("Nenhum dado consolidado (sem colunas de valor detectadas).")

    base = pd.concat(out_parts, ignore_index=True)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    base.to_csv(out_path, index=False, sep=OUT_SEP, encoding=OUT_ENCODING)
    print(f"✅ Base consolidada salva em: {out_path} (linhas={len(base)})")
    return base


In [8]:
base = gerar_base_consolidada(OUT_PROCESSADO, OUT_BASE_CONSOLIDADA)
base.head()

Processados encontrados: 8587 em C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\Indicadores_processado_por_tema


Consolidando:   0%|          | 1/8587 [00:00<1:21:01,  1.77it/s]C:\Users\luiz.felipe\AppData\Local\Temp\ipykernel_58780\3812103745.py:55: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  melted = pd.melt(df, id_vars=id_cols, value_vars=value_cols, var_name="variavel", value_name="valor")
Consolidando:   0%|          | 7/8587 [00:11<5:24:47,  2.27s/it]C:\Users\luiz.felipe\AppData\Local\Temp\ipykernel_58780\3812103745.py:55: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  melted = pd.melt(df, id_vars=id_cols, value_vars=value_cols, var_name="variavel", value_name="valor")
Consolidando:   0%|          | 8/8587 [00:11<4:10:58

✅ Base consolidada salva em: C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\outputs\base_consolidada_tsbio.csv (linhas=3703831)


,indicador_id,tema,categoria,dimensao_tdr,fonte,recorte,variavel,valor,valor_num,unidade,ano,classe,territorio_id,territorio_nome,cod_municipio,municipio_nome,sigla_uf,arquivo_origem
0,agropecuaria__ibge_pam_2024__lavoura_permanente,Lavoura Permanente,Agropecuária,produtiva,IBGE PAM 2024,Municípios,linhas,125970,125970.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agropecuaria__ibge_pam_2024__lavoura_temporaria,Lavoura Temporaria,Agropecuária,produtiva,IBGE PAM 2024,Municípios,linhas,109395,109395.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,agropecuaria__ibge_pevs_2024__producao_extraca...,Produção Extração Vegetal,Agropecuária,produtiva,IBGE PEVS 2024,BR,linhas,10471,10471.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,agropecuaria__ibge_pevs_2024__producao_silvicu...,Produção Silvicultura,Agropecuária,produtiva,IBGE PEVS 2024,BR,linhas,69,69.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,agropecuaria__ibge_ppm_2024__efetivo_rebanhos__br,Efetivo Rebanhos,Agropecuária,produtiva,IBGE PPM 2024,BR,linhas,15838,15838.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3) Gerar catálogo de indicadores (CSV + XLSX)

O catálogo é uma tabela “dimensão” que complementa a base consolidada e ajuda no Looker Studio.


In [9]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

def identificar_colunas_valor_cols(cols: List[str]) -> List[str]:
    excluir = {
        "indicador_id","territorio_id","territorio_nome","categoria","fonte","tema","recorte","dimensao_tdr",
        "cod_municipio","municipio_nome","sigla_uf","ano","mes","classe","observacao","arquivo_origem",
    }
    return [c for c in cols if c not in excluir]

def inferir_unidade(variaveis: List[str]) -> str:
    units = set()
    for v in variaveis:
        suf = str(v).split("_")[-1].lower()
        u = UNIT_SUFFIX_TO_UNIT.get(suf, "")
        if u:
            units.add(u)
    if len(units) == 1:
        return list(units)[0]
    if len(units) > 1:
        return "multiplas"
    return ""

def inferir_periodo(path: Path) -> str:
    try:
        df = read_csv_local(path)
        if "ano" not in df.columns:
            return ""
        anos = pd.to_numeric(df["ano"], errors="coerce").dropna()
        if anos.empty:
            return ""
        return f"{int(anos.min())}-{int(anos.max())}"
    except Exception:
        return ""

def inferir_frequencia_por_fonte(fonte: str) -> str:
    f = (fonte or "").lower()
    if "censo" in f:
        return "decenal"
    if "mapbiomas" in f or "inpe" in f:
        return "anual"
    if "rais" in f or "caged" in f:
        return "mensal/anual"
    return "a definir"

def criar_excel(path_xlsx: Path, df: pd.DataFrame, sheet_name: str = "catalogo"):
    wb = Workbook()
    ws = wb.active
    ws.title = sheet_name

    header_fill = PatternFill("solid", fgColor="1F4E79")
    header_font = Font(color="FFFFFF", bold=True)
    center = Alignment(horizontal="center", vertical="center", wrap_text=True)
    left = Alignment(horizontal="left", vertical="top", wrap_text=True)
    thin = Side(style="thin", color="D9D9D9")
    border = Border(left=thin, right=thin, top=thin, bottom=thin)

    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=1):
        ws.append(row)
        if r_idx == 1:
            for c_idx in range(1, len(row) + 1):
                cell = ws.cell(row=r_idx, column=c_idx)
                cell.fill = header_fill
                cell.font = header_font
                cell.alignment = center
                cell.border = border
        else:
            for c_idx in range(1, len(row) + 1):
                cell = ws.cell(row=r_idx, column=c_idx)
                cell.alignment = left
                cell.border = border

    for col in ws.columns:
        max_len = 0
        col_letter = col[0].column_letter
        for cell in col:
            try:
                max_len = max(max_len, len(str(cell.value)))
            except Exception:
                pass
        ws.column_dimensions[col_letter].width = min(max(12, max_len + 2), 55)

    wb.save(path_xlsx)

def gerar_catalogo(root_processado: Path, out_csv: Path, out_xlsx: Path) -> pd.DataFrame:
    files = sorted(root_processado.rglob("*.csv"))
    registros = []

    for p in tqdm(files, desc="Catalogando"):
        categoria = p.parent.name

        try:
            df_head = read_csv_local(p).head(5)
        except Exception:
            continue

        indicador_id = str(df_head["indicador_id"].dropna().iloc[0]) if "indicador_id" in df_head.columns and df_head["indicador_id"].notna().any() else ""
        tema = str(df_head["tema"].dropna().iloc[0]) if "tema" in df_head.columns and df_head["tema"].notna().any() else ""
        fonte = str(df_head["fonte"].dropna().iloc[0]) if "fonte" in df_head.columns and df_head["fonte"].notna().any() else ""
        recorte = str(df_head["recorte"].dropna().iloc[0]) if "recorte" in df_head.columns and df_head["recorte"].notna().any() else ""
        dimensao_tdr = str(df_head["dimensao_tdr"].dropna().iloc[0]) if "dimensao_tdr" in df_head.columns and df_head["dimensao_tdr"].notna().any() else CATEGORIA_TO_DIMENSAO.get(categoria, "outros")

        variaveis = identificar_colunas_valor_cols(list(df_head.columns))
        unidade = inferir_unidade(variaveis)
        periodo = inferir_periodo(p)
        freq = inferir_frequencia_por_fonte(fonte)

        registros.append({
            "indicador_id": indicador_id,
            "categoria": categoria,
            "tema": tema,
            "fonte": fonte,
            "recorte": recorte,
            "dimensao_tdr": dimensao_tdr,
            "unidade": unidade,
            "nivel_territorial": "municipio",
            "periodo": periodo,
            "frequencia_update": freq,
            "anexo_ii": "não",  # marcar depois (ou criar regras)
            "arquivo_csv": str(p),
            "variaveis": ", ".join(variaveis[:80]),
            "n_variaveis": len(variaveis),
        })

    cat = pd.DataFrame(registros).sort_values(["categoria", "fonte", "tema"])
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    cat.to_csv(out_csv, index=False, encoding=OUT_ENCODING)
    criar_excel(out_xlsx, cat)

    print(f"✅ Catálogo salvo em: {out_csv}")
    print(f"✅ Catálogo (XLSX) salvo em: {out_xlsx}")
    return cat


In [10]:
catalogo = gerar_catalogo(OUT_PROCESSADO, OUT_CATALOGO_CSV, OUT_CATALOGO_XLSX)
catalogo.head()

Catalogando: 100%|██████████| 8587/8587 [02:09<00:00, 66.45it/s] 


✅ Catálogo salvo em: C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\outputs\catalogo_indicadores_tsbio.csv
✅ Catálogo (XLSX) salvo em: C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitHub\fas_tsbio\data\outputs\catalogo_indicadores_tsbio.xlsx


,indicador_id,categoria,tema,fonte,recorte,dimensao_tdr,unidade,nivel_territorial,periodo,frequencia_update,anexo_ii,arquivo_csv,variaveis,n_variaveis
3,,Agropecuária,,,,produtiva,,municipio,2017-2017,a definir,não,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,"Código TSBio, Nome TSBio, Categoria, Tema, Cód...",8
4,,Agropecuária,,,,produtiva,,municipio,,a definir,não,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,"Código TSBio, Nome TSBio, Categoria, Tema, Cód...",9
13,,Agropecuária,,,,produtiva,,municipio,1974-2024,a definir,não,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,"Código TSBio, Nome TSBio, Categoria, Tema, Cód...",13
14,,Agropecuária,,,,produtiva,,municipio,1974-2024,a definir,não,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,"Código TSBio, Nome TSBio, Categoria, Tema, Cód...",13
15,,Agropecuária,,,,produtiva,,municipio,,a definir,não,C:\Users\luiz.felipe\Desktop\FLP\MapiaEng\GitH...,"Código TSBio, Nome TSBio, Categoria, Tema, Cód...",11


## 4) Checagens rápidas (sanidade)

Você pode usar estas checagens antes de publicar no Looker Studio.


In [11]:
print("Base consolidada:")
print(" - linhas:", len(base))
print(" - indicadores:", base["indicador_id"].nunique() if "indicador_id" in base.columns else None)
print(" - municipios:", base["cod_municipio"].nunique() if "cod_municipio" in base.columns else None)
print(" - anos:", (int(base["ano"].min()), int(base["ano"].max())) if "ano" in base.columns and pd.api.types.is_numeric_dtype(base["ano"]) else None)

print("\nCatálogo:")
print(" - linhas:", len(catalogo))
print(" - indicadores:", catalogo["indicador_id"].nunique() if "indicador_id" in catalogo.columns else None)

# Top 10 indicadores com mais linhas
if "indicador_id" in base.columns:
    base.groupby("indicador_id").size().sort_values(ascending=False).head(10)

Base consolidada:
 - linhas: 3703831
 - indicadores: 7914
 - municipios: 65
 - anos: None

Catálogo:
 - linhas: 8587
 - indicadores: 7915


## 5) Publicar no Looker Studio (recomendação)

- Fonte 1: `outputs/base_consolidada_tsbio.csv`
- Fonte 2: `outputs/catalogo_indicadores_tsbio.xlsx` (ou CSV)
- Relacionar por: `indicador_id`

> Dica: Use filtros por `territorio_nome`, `ano`, `dimensao_tdr` e `fonte`.
